In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas
import folium
from countryinfo import CountryInfo
import requests
import seaborn as sns

In [26]:
purities = pd.read_csv("datasets/purities.csv", delimiter=",", decimal=",")

purities = purities.replace("Türkiye", "Turkey")
purities = purities.replace("Iran (Islamic Republic of)", "Iran")
purities = purities.replace("Bolivia (Plurinational State of)", "Bolivia")
purities = purities.replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom")
purities = purities.replace("Bahamas", "The Bahamas")
purities = purities.replace("Venezuela (Bolivarian Republic of)", "Venezuela")
purities = purities.replace("Russian Federation", "Russia")
purities = purities.replace("Côte d'Ivoire", "Ivory Coast")
purities = purities.replace("Lao People's Democratic Republic", "Laos")
purities = purities.replace("Republic of Korea", "South Korea")
purities = purities.replace("Viet Nam", "Vietnam")
purities = purities.replace("Serbia", "Republic of Serbia")
purities = purities.replace("Republic of Moldova", "Moldova")
purities = purities.replace("Syrian Arab Republic", "Syria")
purities = purities.replace("China, Macao SAR", "China")
purities = purities.replace("China, Hong Kong SAR", "Hong Kong")
purities = purities.replace("China, Taiwan Province of China", "Taiwan")
purities = purities.replace("Timor-Leste", "East Timor")
purities = purities.replace("North Macedonia", "Macedonia")
purities = purities.replace("Czechia", "Czech Republic")
purities = purities.replace("State of Palestine", "Israel") # giga sadeg

purities['Typical purity'] = pd.to_numeric(purities['Typical purity'], errors='coerce')
purities['Maximum purity'] = pd.to_numeric(purities['Maximum purity'], errors='coerce')
purities['Minimum purity'] = pd.to_numeric(purities['Minimum purity'], errors='coerce')

purities = purities[purities['Drug'] == 'Cocaine']
def przefiltruj_nulle_df(df, kolumna1, kolumna2, kolumna3):
    filtered_df = df[(~df[kolumna1].isnull()) | (~df[kolumna2].isnull() & ~df[kolumna3].isnull())]
    return filtered_df

def podmien_null_na_srednia(df, kolumna_z_null, kolumna1, kolumna2):
    df[kolumna_z_null].fillna((df[kolumna1] + df[kolumna2]) / 2, inplace=True)
    return df

purities = przefiltruj_nulle_df(purities, 'Typical purity', 'Maximum purity', 'Minimum purity')
purities = podmien_null_na_srednia(purities, 'Typical purity', 'Maximum purity', 'Minimum purity')

In [27]:
def filtrowanie(df, kolumna, wartosc):
    filtered_df = df[df[kolumna] == wartosc]
    return filtered_df

streetPurities = filtrowanie(purities, 'LevelOfSale', 'Street')
wholesalePurities = filtrowanie(purities, 'LevelOfSale', 'Wholesale')


In [28]:
def mapka(df, legend):
    map = folium.Map(location=[50, 10], zoom_start=4)
    df_world = geopandas.read_file("datasets/110m_cultural.zip").merge(
        df,
        how='right',
        left_on=['ADMIN'],
        right_on=['Country/Territory']
    )
    folium.Choropleth(
        geo_data=df_world,
        data=df_world,
        columns=['Country/Territory', 'Typical purity'],
        key_on="feature.properties.Country/Territory",
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=legend
    ).add_to(map)
    return map

wholesalePuritiesAverage = wholesalePurities.groupby('Country/Territory')['Typical purity'].mean()

mapka(wholesalePuritiesAverage.reset_index(), '"Czystość" kokainy w % w przypadku sprzedarzy hurtowej')

In [29]:
streetPuritiesAverage = streetPurities.groupby('Country/Territory')['Typical purity'].mean()
df = streetPuritiesAverage.reset_index()
mapka(df[df['Country/Territory'] != 'Belgium'], '"Czystość" kokainy w % w przypadku sprzedarzy detalicznej')